In [1]:
!pip install requests
import requests
import json
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

def float_to_string(f):
  f = np.round(f,4)
  s = str(f)[2:]
  while len(s) < 4:
    s += "0"
  return s

In [133]:
end_reached = False
year='2019'
game_number = .0001
list_2018 = []

while not end_reached:

  try:
    game_string = float_to_string(game_number)
    request_string = f"https://statsapi.web.nhl.com/api/v1/game/{year}02{game_string}/boxscore"
    response = requests.get(request_string)
  except:
    #Reached the end of the season
    end_reached = True
    break

  game = json.loads(response.content.decode('utf-8'))
  teams=['away', 'home']

  for team in teams:
    try:
      players = list(game['teams'][team]['players'].keys())
    except:
      end_reached = True
      print(game_number, game_string)
      break
    stats = []
    for player in players:
      try:
        game['teams'][team]['players'][player]['stats']['skaterStats']['player_id'] = player
        stats.append(game['teams'][team]['players'][player]['stats']['skaterStats'])
      except:
        try:
          #Player is a goalie or is scratched
          game['teams'][team]['players'][player]['stats']['goalieStats']['player_id'] = player
          stats.append(game['teams'][team]['players'][player]['stats']['goalieStats'])
        except:
          #Player scratched
          continue
    df = pd.DataFrame.from_dict(stats)
    df = df.set_index('player_id')
    df[f'{team}_team_name'] = game['teams'][team]['team']['name']
    df[f'{team}_team_id'] = game['teams'][team]['team']['id']
    df['season'] = year
    df['game_number'] = game_number
    list_2018.append(df)
  game_number += .0001

KeyError: ignored

In [ ]:
df_2018 = pd.concat(list_2018)

In [ ]:
df_2018.sample(10)

In [ ]:
len(df_2018)

In [132]:
df_2018.to_csv('/content/drive/MyDrive/2019_box_scores.csv')

In [161]:
from numpy.core.numeric import NaN
home_teams = df_2018[df_2018['away_team_name'].isna()]
away_teams = df_2018[df_2018['home_team_name'].isna()]
by_game = away_teams.groupby(['game_number']).sum()

In [163]:
by_game.head()

,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,faceOffPct,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,away_team_id,home_team_id,shortHandedSavePercentage
game_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0001,6,3,62,18.0,1.0,2.0,6.0,37.0,63.0,5.0,21.0,0.0,0.0,24.0,3.0,397.23,0.0,34.0,5.0,0.0,29.0,0.0,30.0,6.0,94.444444,83.333333,96.666667,0.0,190.0,0.0
0.0002,12,7,62,16.0,4.0,5.0,14.0,19.0,60.0,13.0,9.0,0.0,0.0,15.0,15.0,344.45,0.0,25.0,2.0,3.0,20.0,3.0,20.0,2.0,100.000000,100.000000,100.000000,0.0,285.0,100.0
0.0003,9,5,58,16.0,0.0,0.0,19.0,25.0,59.0,7.0,8.0,0.0,0.0,18.0,15.0,362.83,0.0,33.0,6.0,2.0,25.0,2.0,27.0,6.0,94.285714,100.000000,92.592593,0.0,437.0,100.0
0.0004,1,2,47,21.0,0.0,0.0,6.0,31.0,60.0,10.0,8.0,1.0,0.0,13.0,-7.0,490.28,0.0,10.0,0.0,0.0,10.0,0.0,12.0,2.0,71.428571,0.000000,83.333333,0.0,532.0,0.0
0.0005,0,0,57,25.0,0.0,0.0,4.0,38.0,67.0,1.0,18.0,0.0,0.0,8.0,-16.0,530.09,0.0,22.0,0.0,1.0,21.0,1.0,23.0,1.0,88.000000,0.000000,91.304348,0.0,133.0,100.0


In [154]:
by_game.head()

,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,faceOffPct,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,away_team_id,home_team_id,shortHandedSavePercentage
game_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0001,10,5,124,52.0,2.0,4.0,12.0,63.0,126.0,10.0,31.0,0.0,0.0,40.0,0.0,701.65,0.0,57.0,8.0,0.0,49.0,0.0,52.0,10.0,182.905983,158.333333,187.575758,152.0,190.0,0.0
0.0002,12,7,124,44.0,4.0,5.0,46.0,60.0,120.0,18.0,12.0,0.0,0.0,27.0,0.0,732.78,0.0,55.0,8.0,7.0,40.0,7.0,43.0,12.0,262.573099,208.333333,274.166667,120.0,285.0,200.0
0.0003,13,7,115,33.0,0.0,0.0,26.0,59.0,118.0,8.0,12.0,0.0,0.0,24.0,-1.0,720.16,0.0,51.0,9.0,4.0,38.0,4.0,44.0,9.0,176.103896,200.000000,169.063181,380.0,437.0,200.0
0.0004,11,7,95,46.0,2.0,4.0,12.0,60.0,120.0,19.0,14.0,1.0,0.0,29.0,-2.0,782.21,0.0,41.0,1.0,0.0,40.0,1.0,43.0,3.0,165.367965,100.000000,180.107527,456.0,532.0,0.0
0.0005,7,4,115,38.0,1.0,2.0,10.0,67.0,134.0,2.0,24.0,1.0,1.0,21.0,-2.0,760.19,0.0,54.0,2.0,1.0,51.0,1.0,53.0,3.0,188.000000,100.000000,191.304348,114.0,133.0,100.0


In [148]:
by_game_away = pd.DataFrame(df_2018.groupby(['game_number'])).groupby(['away_team_name']).sum()
by_game_home = pd.DataFrame(df_2018.groupby(['game_number'])).groupby(['home_team_name']).sum()
by_game = pd.concat([by_game_away, by_game_home])
by_game.head(10)
#Shots are wrong...it's the whole game; divide by 2
#game_number is summed
#use sum of goalie time on ice to figure out if it went to over time; need this for good predictions
#faceoff % is summed
#pim is probably wrong


KeyError: ignored

In [146]:
df_2018.sample(10)

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,powerPlayTimeOnIce,shortHandedTimeOnIce,faceOffPct,pim,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage,away_team_name,away_team_id,season,game_number,home_team_name,home_team_id,shortHandedSavePercentage
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ID8478472,11:14,0,1,5,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,11:03,0:11,0:00,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0325,Tampa Bay Lightning,14.0,NaN
ID8477935,12:20,0,0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,11:00,1:20,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.1008,Calgary Flames,20.0,NaN
ID8476881,21:49,0,0,0,0.0,0.0,0.0,0.0,14.0,22.0,2.0,1.0,0.0,0.0,1.0,-1.0,14:53,4:19,2:37,63.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0902,San Jose Sharks,28.0,NaN
ID8474066,14:48,0,0,0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,14:18,0:00,0:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0969,New York Islanders,2.0,NaN
ID8476925,17:36,0,0,3,1.0,0.0,0.0,0.0,6.0,12.0,0.0,0.0,0.0,0.0,1.0,0.0,10:26,4:59,2:11,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0202,Nashville Predators,18.0,NaN
ID8476981,17:55,0,1,4,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16:27,0:50,0:38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columbus Blue Jackets,29.0,2018,0.0900,NaN,NaN,NaN
ID8476882,19:44,1,2,2,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,15:55,2:24,1:25,50.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carolina Hurricanes,12.0,2018,0.0823,NaN,NaN,NaN
ID8476443,15:51,0,0,2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,-2.0,14:56,0:00,0:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,0.0051,Montréal Canadiens,8.0,NaN
ID8477401,5:55,0,0,1,1.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5:55,0:00,0:00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago Blackhawks,16.0,2018,0.1266,NaN,NaN,NaN
